In [1]:
import modal.cli.volume
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import imageio
import os
from pathlib import Path
from tqdm.auto import tqdm
from typing import Literal
from IPython.display import HTML
from loguru import logger

ProjectionType = Literal["Orthographic", "PlateCarree", "Mollweide", "Robinson"]

def generate_rollout_gif(
    output_paths: list[str | Path],
    channel: str,
    output_dir: str | Path,
    cmap: str = "coolwarm",
    projection: str = "Orthographic",
):
    """
    Creates a GIF from a list of NetCDF files using xarray and Cartopy for plotting.
    Each file is visualized on a 3D-like globe using a configurable projection.

    Parameters:
    - output_paths: List of paths to the output NetCDF files.
    - channel: Name of the data channel to plot (e.g., 't2m').
    - output_dir: Directory path where the GIF should be saved.
    - cmap: Colormap to use for the plots.
    - projection: The Cartopy projection to use for plotting.

    # Example usage:
    generate_rollout_gif(output_paths = output_paths,
                         channel='t2m',
                         output_dir='.',
                         projection='Orthographic')
    """

    # Initialize lists to store all data for computing global statistics
    all_data = []

    # Load data and append to list
    for file_path in output_paths:
        with xr.open_dataarray(file_path) as ds:
            data = ds.sel(channel=channel).isel(time=-1).squeeze()
            all_data.append(data.values.flatten())

    # Concatenate all data into a single array and compute global statistics
    all_data = np.concatenate(all_data)
    mean_val = np.mean(all_data)
    std_val = np.std(all_data)
    global_min = mean_val - 3 * std_val
    global_max = mean_val + 3 * std_val

    # create gif_path including model name and simulation time span
    model_name = os.path.basename(output_paths[0]).split("__")[0]
    start_time = os.path.basename(output_paths[0]).split("__")[2]
    end_time = os.path.basename(output_paths[-1]).split("__")[3].replace(".nc", "")
    gif_path = (
        Path(output_dir) / f"{model_name}_{start_time}_to_{end_time}_{channel}.gif"
    )

    # Check if the gif already exists
    if gif_path.exists():
        print(f"GIF already exists: {gif_path}")
        return gif_path

    images = []
    for file_path in tqdm(output_paths, desc="Generating GIF Frames"):
        ds = xr.open_dataarray(file_path)
        data = ds.sel(channel=channel).isel(time=-1).squeeze()

        projection_class = getattr(ccrs, projection)
        plt.figure(figsize=(10, 10))
        ax = plt.axes(
            projection=projection_class(central_longitude=0, central_latitude=0)
        )
        ax.add_feature(cfeature.COASTLINE)
        ax.add_feature(cfeature.BORDERS, linestyle=":")

        img = data.plot(
            ax=ax,
            transform=ccrs.PlateCarree(),
            cmap=cmap,
            add_colorbar=False,
            vmin=global_min,
            vmax=global_max,
        )
        plt.colorbar(
            img, ax=ax, orientation="vertical", pad=0.02, aspect=50, extend="both"
        )
        ax.set_title(f"{channel.upper()} at Time Step: {ds.time[-1].values}")

        temp_png_path = f"temp_{os.path.basename(file_path)}.png"
        plt.savefig(temp_png_path, bbox_inches="tight", dpi=300)
        plt.close()

        images.append(imageio.imread(temp_png_path))
        os.remove(temp_png_path)

    imageio.mimsave(gif_path, images, duration=1, loop=0)
    print(f"GIF saved to {gif_path}")
    return gif_path


def visualize_rollout(
    output_paths: list[str | Path],
    channels: str,
    output_dir: str | Path,
    cmap="coolwarm",
    projection="Orthographic",
):
    """
    Generates and displays GIFs for specified variables from NetCDF files.

    Parameters:
    - output_paths: List of paths to the output NetCDF files.
    - channels: List of variable names to plot and generate GIFs.
    - cmap: Colormap to use for the plots.
    - projection: The Cartopy projection to use for plotting.
    """
    gif_paths = []
    for channel in channels:
        gif_path = generate_rollout_gif(
            output_paths=output_paths,
            channel=channel,
            output_dir=output_dir,
            cmap=cmap,
            projection=projection,
        )
        gif_paths.append(gif_path)
    logger.debug(f"Generated GIFs: {gif_paths}")
    return display_gifs_side_by_side(gif_paths)


def display_gifs_side_by_side(gif_paths):
    """Display given GIFs side by side in a Jupyter Notebook."""
    if not gif_paths:
        return HTML("<p>No GIFs provided.</p>")

    num_gifs = len(gif_paths)
    width_percent = 100 / num_gifs

    img_tags = " ".join(
        f'<img src="{path}" style="width: {width_percent}%; margin-right: 2px;">'
        for path in gif_paths[:-1]
    )
    img_tags += f'<img src="{gif_paths[-1]}" style="width: {width_percent}%;">'  # Last image without right margin

    html_str = f"""
    <div style="display:flex; justify-content:space-between;">
        {img_tags}
    </div>
    """
    return HTML(html_str)


# Example usage:
# output_paths = ['path_to_netCDF1.nc', 'path_to_netCDF2.nc']
# channels = ['t2m', 'u10m', 'v10m']
# visualize_rollout(output_paths, channels)


/home/mateuscorrea/miniconda3/envs/skyenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
import os

def listar_arquivos_na_pasta(caminho_da_pasta):
    """
    Lista todos os arquivos dentro de um caminho de pasta especificado.

    Args:
        caminho_da_pasta (str): O caminho da pasta a ser inspecionada.

    Returns:
        list: Uma lista contendo os caminhos completos de todos os arquivos na pasta.
              Retorna uma lista vazia se a pasta não existir ou não contiver arquivos.
    """
    lista_de_arquivos = []
    if os.path.isdir(caminho_da_pasta):
        for nome_arquivo in os.listdir(caminho_da_pasta):
            caminho_completo = os.path.join(caminho_da_pasta, nome_arquivo)
            if os.path.isfile(caminho_completo):
                lista_de_arquivos.append(caminho_completo)
    return lista_de_arquivos

# Exemplo de uso:
# Substitua 'results/CjNbe1uDsF' pelo caminho da sua pasta real
caminho_da_pasta = 'results/'
arquivos_encontrados = listar_arquivos_na_pasta(caminho_da_pasta)

if arquivos_encontrados:
    print(f"Arquivos encontrados na pasta '{caminho_da_pasta}':")
    for arquivo in arquivos_encontrados:
        print(arquivo)
else:
    print(f"Nenhum arquivo encontrado ou a pasta '{caminho_da_pasta}' não existe.")


Arquivos encontrados na pasta 'results/':
results/pangu__file__20250620_12_00__20250620_18_00.nc
results/pangu__gfs__20250620_00_00__20250620_06_00.nc
results/pangu__file__20250622_06_00__20250622_12_00.nc
results/pangu__file__20250621_12_00__20250621_18_00.nc
results/pangu__file__20250621_06_00__20250621_12_00.nc
results/pangu__file__20250622_18_00__20250623_00_00.nc
results/pangu__file__20250621_00_00__20250621_06_00.nc
results/pangu__file__20250622_00_00__20250622_06_00.nc
results/pangu__file__20250620_18_00__20250621_00_00.nc
results/pangu__file__20250622_12_00__20250622_18_00.nc
results/pangu__file__20250621_18_00__20250622_00_00.nc
results/pangu__file__20250620_06_00__20250620_12_00.nc


In [22]:
visualize_rollout(output_paths=arquivos_encontrados, channels=["u10m", "v10m"],output_dir=".")

Generating GIF Frames:   0%|          | 0/12 [00:00<?, ?it/s]/tmp/ipykernel_67858/3573539834.py:100: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_png_path))
Generating GIF Frames:   8%|▊         | 1/12 [00:01<00:20,  1.89s/it]/tmp/ipykernel_67858/3573539834.py:100: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_png_path))
Generating GIF Frames:  17%|█▋        | 2/12 [00:03<00:18,  1.87s/it]/tmp/ipykernel_67858/3573539834.py:100: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to

GIF saved to pangu_20250620_06_00_to_20250620_06_00_u10m.gif


Generating GIF Frames:   0%|          | 0/12 [00:00<?, ?it/s]/tmp/ipykernel_67858/3573539834.py:100: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_png_path))
Generating GIF Frames:   8%|▊         | 1/12 [00:01<00:20,  1.83s/it]/tmp/ipykernel_67858/3573539834.py:100: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_png_path))
Generating GIF Frames:  17%|█▋        | 2/12 [00:03<00:18,  1.88s/it]/tmp/ipykernel_67858/3573539834.py:100: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to

GIF saved to pangu_20250620_06_00_to_20250620_06_00_v10m.gif


In [29]:
import xarray as xr

ds = xr.open_dataset("pangu__gfs__20250620_00:00__20250620_06:00.nc")
print(ds)


<xarray.Dataset> Size: 573MB
Dimensions:     (time: 2, channel: 69, lat: 721, lon: 1440)
Coordinates:
  * channel     (channel) <U5 1kB 'z1000' 'z925' 'z850' ... 'u10m' 'v10m' 't2m'
  * lat         (lat) float64 6kB 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * lon         (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time        (time) datetime64[ns] 16B 2025-06-20 2025-06-20T06:00:00
Data variables:
    prediction  (time, channel, lat, lon) float32 573MB ...


In [33]:
ds.prediction

<xarray.DataArray 'prediction' (time: 2, channel: 69, lat: 721, lon: 1440)> Size: 573MB
array([[[[ 814.0479 , ...,  814.0479 ],
         ...,
         [2339.2283 , ..., 2339.2283 ]],

        ...,

        [[ 273.1848 , ...,  273.1848 ],
         ...,
         [ 218.68481, ...,  218.68481]]],


       [[[ 722.57434, ...,  717.7853 ],
         ...,
         [1217.1675 , ..., 1185.6405 ]],

        ...,

        [[ 273.1052 , ...,  273.1189 ],
         ...,
         [ 242.35571, ...,  244.3429 ]]]], dtype=float32)
Coordinates:
  * channel  (channel) <U5 1kB 'z1000' 'z925' 'z850' ... 'u10m' 'v10m' 't2m'
  * lat      (lat) float64 6kB 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * lon      (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time     (time) datetime64[ns] 16B 2025-06-20 2025-06-20T06:00:00

In [11]:
from modal import Volume
from modal import cli
volume = Volume.from_name("forecasts", create_if_missing=False)
# Lista todos os arquivos no volume
file_list = volume.listdir("/CjNbe1uDsF")

# Baixa um por um para a pasta local "./results/"
for filename in file_list:
    remote_path = f"/CjNbe1uDsF/{filename}"
    local_path = f"./results/{filename}"
    print(f"Baixando {remote_path} -> {local_path}")
    cli.volume.get("forecasts", remote_path, local_path)


Baixando /CjNbe1uDsF/FileEntry(path='CjNbe1uDsF/pangu__file__20250622_18:00__20250623_00:00.nc', type=<FileEntryType.FILE: 1>, mtime=1750529716, size=573126688) -> ./results/FileEntry(path='CjNbe1uDsF/pangu__file__20250622_18:00__20250623_00:00.nc', type=<FileEntryType.FILE: 1>, mtime=1750529716, size=573126688)


AttributeError: module 'modal.cli' has no attribute 'volume'